# Geoscripting Exercise 12 GISRSArtists
## 22 January 2019

Imports

In [1]:
from geopy.geocoders import Nominatim
from owslib.wcs import WebCoverageService
from python.createBoundingBox import createBoundingBox
from python.writeGeotiffToFile import writeGeotiffToFile
from python.getGeotiffFromWebCoverageService import getGeotiffFromWebCoverageService
from python.getDataFromWebFeatureService import getDataFromWebFeatureService
from python.calculateBuildingVolume import calculateBuildingVolume
from python.visualizeBuildingVolume import visualizeBuildingVolume
from python.geocodePlacenameToCoordinates import geocodePlacenameToCoordinates

ModuleNotFoundError: No module named 'geopy'

In [ ]:
x, y = 174100, 444100